In [28]:
import pandas as pd
import web3

In [30]:
df = pd.read_csv("data.csv")
all_wallet_addresses = set()
cols = df.columns
for col in cols:
    for wallet_address in df[col].dropna():
        wallet_address = w3.to_checksum_address(wallet_address)
        all_wallet_addresses.add(wallet_address)

In [31]:
len(all_wallet_addresses)

1773

In [43]:
rpc_url = "https://mainnet.infura.io/v3/c4120217bf7f4dd18189de1e986ff8c1"
provider = web3.providers.rpc.HTTPProvider(rpc_url)
w3 = web3.Web3(provider)

# abi that exposes IERC20.balanceOf method
abi = [
    {
        "inputs": [{"internalType": "address", "name": "account", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function",
    }
]

token_addresses = [
    ("OgAndy", "0x748509433ef209c4d11ada51347d5724a5da0ca5"),
    ("SLERF", "0x7f1fe3623c60195233782ee390f98be7e4ed2b47"),
    ("BRANDY", "0xf00d980b525e7cfe351259a37efb72f077396eca"),
]

balances = pd.DataFrame(columns=["Address", "OgAndy", "SLERF", "BRANDY"])
balances.set_index("Address", inplace=True)

num_addresses = 50
# total requests is num_addresses * len(token_addresses)

for token, token_address in token_addresses:
    token_address = w3.to_checksum_address(token_address)
    contract = w3.eth.contract(address=token_address, abi=abi)
    for wallet_address in list(all_wallet_addresses)[:num_addresses]:
        balance = contract.functions.balanceOf(wallet_address).call()
        balances.loc[wallet_address, token] = balance
        print(".", end="")

......................................................................................................................................................

In [44]:
balances

,OgAndy,SLERF,BRANDY
Address,,,
0x6241bDA1314853274eE1E4745a83695dCce2F80a,0,0,0
0x7982b9096D64ABD4fDAb241AB9Bac75b6F9F3412,0,0,0
0xa84835d0b0f0001539b2F251f134Bb3B180082CE,0,0,0
0x0F06af5B03F3308F6ecF163E7974b73e8BC3372a,0,0,0
0x2FC3b3C50957F590148314Aaffa92E922C2181EA,420133291070,0,0
0x08da663Aa0856e1B368d877C5989e9c646Ee23B7,0,0,0
0xb4Ec142134c0CFa59486EAc017e5d6102226D185,351385935,0,0
0xc7655e14416d97c4F9eF701DCD03aCAfA0c532CE,0,0,17933175591804143
0xD58c5f91b4fbb6074105F167E0B32bCA0faB5Ab1,835145080889,0,0
